In [1]:
# Import Relevant Modules
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
# pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, split

In [2]:
# Initialize Spark Session
spark = SparkSession.builder.appName("Spotify Analysis Spark Session").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/05 20:53:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load Data Using Spark
spotify_data = (spark.read.option("header", "true").csv('data/Final database.csv'))
# Pandas Method --> spotify_data = pd.read_csv('data/Final database.csv')

In [4]:
# Secondary Feature 1: Lyric Mood Score -------------------------------------------------------------------
# Scale: 1 - 8, where 1 is "low" mood and 8 is "high" mood. -----------------------------------------------
# ---------------------------------------------------------------------------------------------------------

# Create Conditions: Map Feelings/Mood to numerical value
feelings_conditions = when(col("Thug") == 1, 1)\
                    .when(col("Hope") == 1, 2)\
                    .when(col("Desire") == 1, 3)\
                    .when(col("Explore") == 1, 4)\
                    .when(col("Nostalgia") == 1, 5)\
                    .when(col("Love") == 1, 6)\
                    .when(col("Fun") == 1, 7)\
                    .when(col("Celebrate") == 1, 8)\
                    .otherwise(0)

# Map Feelings to Numerical Value: Use Above As Dictionary
spotify_data = spotify_data.withColumn('lyric_mood_score', feelings_conditions)

In [5]:
# Secondary Feature 2: Song Positvity Feature -------------------------------------------------------------
# Binary Encoding of If Song Is Positive of Not -----------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------

spotify_data = spotify_data.withColumn('song_positivity', when(col("positive") >  col("negative"), 1)
                                       .otherwise(0))

In [6]:
# Secondary Feature 3-4: English/Spanish Speaking Country Boolean Feature --------------------------------
# Binary Encoding of If A Song Record Is Shown In (a) an English or (b) A Spanish Speaking Country or Not 
# --------------------------------------------------------------------------------------------------------

spotify_data = spotify_data.withColumn('english_song', when(col("Cluster") == "english speaking and nordic", 1)
                                       .otherwise(0))

spotify_data = spotify_data.withColumn('spanish_song', when(col("Cluster") == "spanish speaking", 1).otherwise(0))

In [7]:
# Secondary Feature 5-7: Date Splitting Date into Year, Month, & Quarter ---------------------------------
# --------------------------------------------------------------------------------------------------------

spotify_data = spotify_data.withColumn('year', split(spotify_data['Release_date'], '-').getItem(0))\
                           .withColumn('month', split(spotify_data['Release_date'], '-').getItem(1))

quarters = when((1 <= col("month")) & (col("month") <= 3), 1)\
          .when((4 <= col("month")) & (col("month") <= 6), 2)\
          .when((7 <= col("month")) & (col("month") <= 9), 3)\
          .when((10 <= col("month")) & (col("month") <= 12), 4)\
          .otherwise(0)

spotify_data = spotify_data.withColumn('quarter', quarters)

In [8]:
# Removed Unwanted Columns From Data
spotify_data = spotify_data\
                            .drop("Album/Single")\
                            .drop("Artist_followers")\
                            .drop("Explicit")\
                            .drop("Track_number")\
                            .drop("Tracks_in_album")\
                            .drop("danceability")\
                            .drop("energy")\
                            .drop("key")\
                            .drop("loudness")\
                            .drop("mode")\
                            .drop("speechiness")\
                            .drop("acoustics")\
                            .drop("instrumentalness")\
                            .drop("liveliness")\
                            .drop("valence")\
                            .drop("tempo")\
                            .drop("duration_ms")\
                            .drop("time_signature")\
                            .drop("Days_since_release")\
                            .drop("Released_after_2017")\
                            .drop("Explicit_false")\
                            .drop("Explicit_true")\
                            .drop("album")\
                            .drop("compilation")\
                            .drop("single")\
                            .drop("syuzhet_norm")\
                            .drop("bing_norm")\
                            .drop("afinn_norm")\
                            .drop("nrc_norm")\
                            .drop("syuzhet")\
                            .drop("bing")\
                            .drop("afinn")\
                            .drop("nrc")\
                            .drop("n_words")\
                            .drop("anger_norm")\
                            .drop("anticipation_norm")\
                            .drop("disgust_norm")\
                            .drop("fear_norm")\
                            .drop("joy_norm")\
                            .drop("sadness_norm")\
                            .drop("surprise_norm")\
                            .drop("trust_norm")\
                            .drop("negative_norm")\
                            .drop("positive_norm")\
                            .drop("anger_norm2")\
                            .drop("anticipation_norm2")\
                            .drop("disgust_norm2")\
                            .drop("fear_norm2")\
                            .drop("joy_norm2")\
                            .drop("sadness_norm2")\
                            .drop("surprise_norm2")\
                            .drop("trust_norm2")\
                            .drop("negative_norm2")\
                            .drop("positive_norm2")\
                            .drop("negative_bog_jr")\
                            .drop("positive_bog_jr")\
                            .drop("Bayes")\
                            .drop("Negative_Bayes")\
                            .drop("Neutral_Bayes")\
                            .drop("Positive_Bayes")\
                            .drop("LDA_Topic")\
                            .drop("bing_norm_negative")\
                            .drop("bing_norm_neutral")\
                            .drop("bing_norm_positive")\
                            .drop("Argentina")\
                            .drop("Australia")\
                            .drop("Austria")\
                            .drop("Belgium")\
                            .drop("Brazil")\
                            .drop("Canada")\
                            .drop("Chile")\
                            .drop("Colombia")\
                            .drop("Costa Rica")\
                            .drop("Denmark")\
                            .drop("Ecuador")\
                            .drop("Finland")\
                            .drop("France")\
                            .drop("Germany")\
                            .drop("Global")\
                            .drop("Indonesia")\
                            .drop("Ireland")\
                            .drop("Italy")\
                            .drop("Malaysia")\
                            .drop("Mexico")\
                            .drop("Netherlands")\
                            .drop("New Zealand")\
                            .drop("Norway")\
                            .drop("Peru")\
                            .drop("Philippines")\
                            .drop("Poland")\
                            .drop("Portugal")\
                            .drop("Singapore")\
                            .drop("Spain")\
                            .drop("Sweden")\
                            .drop("Switzerland")\
                            .drop("Taiwan")\
                            .drop("Turkey")\
                            .drop("UK")\
                            .drop("USA")\
                            .drop("Popu_max")\
                            .drop("Top10_dummy")\
                            .drop("Top50_dummy")\
                            .drop("Cluster")\
                            .drop("bolero")\
                            .drop("boy band")\
                            .drop("country36")\
                            .drop("dance/electronic")\
                            .drop("else")\
                            .drop("funk")\
                            .drop("hip hop")\
                            .drop("house")\
                            .drop("indie")\
                            .drop("jazz")\
                            .drop("k-pop")\
                            .drop("latin")\
                            .drop("metal")\
                            .drop("opm")\
                            .drop("pop")\
                            .drop("r&b/soul")\
                            .drop("rap")\
                            .drop("reggae")\
                            .drop("reggaeton")\
                            .drop("rock")\
                            .drop("trap")\
                            .drop("anticipation")\
                            .drop("anger")\
                            .drop("disgust")\
                            .drop("fear")\
                            .drop("joy")\
                            .drop("sadness")\
                            .drop("surprise")\
                            .drop("trust")\
                            .drop("negative")\
                            .drop("positive")\
                            .drop("Celebrate")\
                            .drop("Desire")\
                            .drop("Explore")\
                            .drop("Fun")\
                            .drop("Hope")\
                            .drop("Love")\
                            .drop("Nostalgia")\
                            .drop("Thug")\
                            .drop("Genre")\
                            .drop("album31")

In [9]:
# Clean DataSet - Deal with N/A Values For Remaining Columns Using Imputation
clean_spotify_data = spotify_data.na.fill(0)

In [10]:
# Rename Columns
clean_spotify_data = clean_spotify_data\
                    .withColumnRenamed("Country0", "Country")\
                    .withColumnRenamed("Uri", "SpotifySongID")\
                    .withColumnRenamed("Album9", "Album")\
                    .withColumnRenamed("Release_date", "ReleaseDate")\
                    .withColumnRenamed("Genre_new", "Genre")\
                    .withColumnRenamed("genre_score", "GenreScore")\
                    .withColumnRenamed("lyric_mood_score", "LyricMoodScore")\
                    .withColumnRenamed("song_positivity", "IsPositiveBool")\
                    .withColumnRenamed("english_song", "IsEnglishBool")\
                    .withColumnRenamed("spanish_song", "IsSpanishBool")\
                    .withColumnRenamed("month", "ReleaseMonth")\
                    .withColumnRenamed("year", "ReleaseYear")\
                    .withColumnRenamed("quarter", "ReleaseQuarter")


# Reorder Columns & Print
clean_spotify_data = clean_spotify_data.select("SpotifySongID", "Title", "Artist", "Album", "Genre", "ReleaseDate",
                                               "ReleaseYear", "ReleaseMonth", "ReleaseQuarter", "Country", 
                                               "IsEnglishBool", "IsSpanishBool", "IsPositiveBool", "Popularity",
                                               "LyricMoodScore")

# Convert Popularity From String to Double
clean_spotify_data = clean_spotify_data.withColumn("Popularity", clean_spotify_data.Popularity.cast('double'))

# Print
clean_spotify_data.printSchema()

root
 |-- SpotifySongID: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Artist: string (nullable = true)
 |-- Album: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- ReleaseDate: string (nullable = true)
 |-- ReleaseYear: string (nullable = true)
 |-- ReleaseMonth: string (nullable = true)
 |-- ReleaseQuarter: integer (nullable = false)
 |-- Country: string (nullable = true)
 |-- IsEnglishBool: integer (nullable = false)
 |-- IsSpanishBool: integer (nullable = false)
 |-- IsPositiveBool: integer (nullable = false)
 |-- Popularity: double (nullable = true)
 |-- LyricMoodScore: integer (nullable = false)



In [11]:
clean_spotify_data.toPandas()

22/08/05 20:53:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album, Release_date, Genre_new, negative, positive, Celebrate, Desire, Explore, Fun, Hope, Love, Nostalgia, Thug, Cluster
 Schema: Country0, Uri, Popularity, Title, Artist, Album9, Release_date, Genre_new, negative, positive, Celebrate, Desire, Explore, Fun, Hope, Love, Nostalgia, Thug, Cluster
Expected: Country0 but found: Country
CSV file: file:///Users/kkissoon/Desktop/UWaterloo%20-%20Final%20Project/data/Final%20database.csv


,SpotifySongID,Title,Artist,Album,Genre,ReleaseDate,ReleaseYear,ReleaseMonth,ReleaseQuarter,Country,IsEnglishBool,IsSpanishBool,IsPositiveBool,Popularity,LyricMoodScore
0,https://open.spotify.com/track/6FyRXC8tJUh863J...,adan y eva,Paulo Londra,Adan y Eva,hip hop,2018-11-05,2018,11,4,Global,0,0,0,31833.95,0
1,https://open.spotify.com/track/6FyRXC8tJUh863J...,adan y eva,Paulo Londra,Adan y Eva,hip hop,2018-11-05,2018,11,4,USA,1,0,0,8.00,0
2,https://open.spotify.com/track/6FyRXC8tJUh863J...,adan y eva,Paulo Londra,Adan y Eva,hip hop,2018-11-05,2018,11,4,Argentina,0,1,0,76924.40,0
3,https://open.spotify.com/track/6FyRXC8tJUh863J...,adan y eva,Paulo Londra,Adan y Eva,hip hop,2018-11-05,2018,11,4,Belgium,1,0,0,849.60,0
4,https://open.spotify.com/track/6FyRXC8tJUh863J...,adan y eva,Paulo Londra,Adan y Eva,hip hop,2018-11-05,2018,11,4,Switzerland,1,0,0,20739.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170628,https://open.spotify.com/track/4v29A51TqtD2KFS...,smalltown,Luke Christopher,SmallTown,pop,2017-07-07,2017,07,3,Taiwan,1,0,0,31.20,0
170629,https://open.spotify.com/track/7InvbALrXpsz58J...,灰狼,G.E.M.,摩天動物園,pop,2019-12-26,2019,12,4,Taiwan,1,0,0,156.00,0
170630,https://open.spotify.com/track/5rCNIuDc3VD7q6H...,預念患者,張赫宣,Freedom,else,2017-06-28,2017,06,2,Taiwan,1,0,0,90.40,0
170631,https://open.spotify.com/track/0525Cll5exsrybt...,清晨巴士,Crowd Lu,有吉他的流行歌曲,indie,2012-11-20,2012,11,4,Taiwan,1,0,0,21.60,0


In [12]:
# Descriptions & Notes
# - Additional Features Engineered Above
# - Further ML Capabilities with these features include:
#     - Predicting Lyric Mood (Secondary Feature 1), given other Features (Artist, Year, Country, Album, etc.)
#     - Ability to predict if a song is in an English or Spanish speaking Country (Secondary Features 2 & 3) given other details
#     - Ability to predict if a song will be deemed as overall "Positive" (Secondary Feature 4) given other details
#     - Date Splitting (Secondary Features 5 - 7) would allow better visulaizations and more precise trends
#     (i.e. artist release predictions and trend forecasts).
    
# Possibble Visualizations Given New Features:
#     - Artist Evolution over time In Respect to lyric-tone.
#     - Geographical Visualization: Porpotion of Spanish/English speaking Song Releases by Country
#     - With Date Split Features: Statistics over time by year, month, quarter, etc.